In [1]:
import os
import pickle
import sys

import copy
import random
import joblib
import numpy as np
import pandas as pd
import rioxarray as rxr
import torch
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

sys.path.append("..")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from src.model_utils import reshape_data
from src.dataprocessing import generate_blocks #generate_subsets, 

## Paths to data

In [2]:
# defining paths
path_to_npys_data = os.path.join("..", "data", "npys_data")

pathTarget = os.path.join(os.path.join(path_to_npys_data, "target_croplands.npy"))
pathFeatures = os.path.join(path_to_npys_data, "features_initial_data.npy")
pathMorf = os.path.join(path_to_npys_data, "features_morf_data.npy")
pathTarget_tif = os.path.join("..", "data", "target", "target_croplands.tif")

##  Data

In [3]:
# Features
climate_features = pd.DataFrame.from_dict(
    np.load(pathFeatures, allow_pickle=True), orient="columns"
)
morf_features = pd.DataFrame.from_dict(
    np.load(pathMorf, allow_pickle=True), orient="columns"
)

climate_keys = list(climate_features.keys())
morf_keys = list(morf_features.keys())

# with open(os.path.join(path_to_npys_data, "climate_keys.pkl"), "wb") as file:
#     pickle.dump(climate_keys, file)

# with open(os.path.join(path_to_npys_data, "morf_keys.pkl"), "wb") as file:
#     pickle.dump(morf_keys, file)

In [4]:
# Target Variable
y = pd.DataFrame.from_dict(np.load(pathTarget, allow_pickle=True), orient="columns")
y = y["Target"].astype(int)
# Set classes 4,5 to 0
y = pd.DataFrame({"target": np.where(y > 3, 0, y)})

## Data Preparation 


### Train/val/test split using pixels blocks

In [5]:
# combine climate morf and target and then filter to make holdout 
climate_features.drop(columns=['latitude', 'longitude'], inplace=True)
data = pd.concat([climate_features, morf_features, y], axis=1)

main_data = data[data['longitude'] <= 100]
hold_out = data[(115 <= data['longitude']) & (data['longitude'] <= 135) &
                     (42 <= data['latitude']) & (data['latitude'] <= 55)]

X_keys = list(data.keys()[:-1])

# with open(os.path.join(path_to_npys_data, "X_keys.pkl"), 'wb') as file:
#     pickle.dump(X_keys, file)

del data

In [6]:
# Calculate the total number of samples in each class
class_counts = np.unique(main_data["target"], return_counts=True)[1]

# Calculate the total number of samples
total_samples = sum(class_counts)
print(class_counts)

[17567497   239918  1266141  2746144]


In [7]:
class_share = [class_count/total_samples for class_count in class_counts]
class_share

[0.805120922835786,
 0.010995476564755703,
 0.05802742475836056,
 0.12585617584109773]

In [8]:
# get nrows and ncols using preprocessed tif
nrows, ncols = (
    rxr.open_rasterio(pathTarget_tif)
    .squeeze()
    .where(rxr.open_rasterio(pathTarget_tif).squeeze()["x"] <= 100, drop=True)
    .shape
)
print(nrows, ncols)

2450 8906


In [9]:
# Reshape features and target dataframes back to its original shape
y = main_data.pop("target").to_numpy().reshape(nrows, ncols)
X = main_data.values.reshape(nrows, ncols, -1)

# # holdout
y_holdout = hold_out.pop("target").to_numpy()
X_holdout = hold_out.to_numpy()

In [10]:
def generate_subsets(
    blocks: list,
    max_iterations: int,
    class_counts: list,
    options: list,
    options_distr: list,
    train_portion: float,
    val_test_portion: float,
    eps: float
):
    """
    Generate subsets of data blocks based on given criteria.

    Args:
        blocks (list): List of data blocks.
        empty (dict): Empty dictionary with keys representing classes and values representing counts.
        max_iterations (int): Maximum number of iterations.
        class_counts (list): List of class counts for each class.
        options (list): List of subsets to generate.
        options_distr (list): List of dictionaries representing class distributions in options.
        train_portion (float): Portion of data for training.
        val_test_portion (float): Portion of data for validation and testing.
        eps (float): Allowed tolerance to violate class distribution

    Returns:
        tuple: A tuple containing updated options, options_distr, and remaining blocks.
    """
    iteration = 0

    while blocks and iteration < max_iterations:
        random.seed(142)            #set the seed 
        random.shuffle(blocks)
        random_element = blocks.pop()

        # Calculate class distribution for the current block
        block_distr = {
            value: count
            for value, count in zip(
                *np.unique(random_element[1].flatten(), return_counts=True)
            )
        }
        block_distr = {key: block_distr.get(key, 0) for key in [0, 1, 2, 3]}

        indexes = list(range(len(options)))   
        
        j_cur = random.randint(0,2)#  train | val | test
        if j_cur==0:
            portion = train_portion
        else:
            portion = val_test_portion

        # Check if adding the block to the subset violates class distribution constraints
        # Current class count
        class_counts_cur = [options_distr[indexes[j_cur]][i] + block_distr[i] for i in range(4)]
        # Current class distribution
        distrib_cur = [amount/sum(class_counts_cur) for amount in class_counts_cur]
        
        # Other classes
        j_other = [0,1,2]
        j_other.remove(j_cur)
        # samples_total = sum(distr_cur) + sum(options_distr[indexes[j_other[0]]].values()) + sum(options_distr[indexes[j_other[1]]].values())
        if all((1-eps)*class_share[i]<=  #Check class stratification
            distrib_cur[i]
            <= (1+eps)*class_share[i] for i in range(4)
        ) & all( class_counts_cur[i] < portion*class_counts[i] for i in [0,1,2,3]): #Check train\test split rule
            # block accepted
            options[indexes[j_cur]].append(random_element)
            # Update class distribution for the selected option
            for key, value in block_distr.items():
                options_distr[indexes[j_cur]][key] += value
        else:
            # block returned
            blocks.append(random_element)

        iteration += 1
        
    # Calculate precisely
    options_distr_share = []
    for sett in options_distr:
        sum_values = sum(sett.values())
        try:
            sett_distr = {key: value/sum_values for key, value in sett.items()}
        except ZeroDivisionError:
            sett_distr = {key: 0 for key in sett.keys()}
        options_distr_share.append(sett_distr)
    print('Final distribution of classes:',options_distr_share)

    return options, options_distr, blocks

In [11]:
minblocks = np.inf
eps = 0.5

#Iterate till train/test/val not empty
for iter in tqdm(range(30)):
    # Generate 100x100 blocks from X and y
    X_blocks = generate_blocks(X, block_size=(100, 100))
    y_blocks = generate_blocks(y, block_size=(100, 100))
    blocks = list(zip(X_blocks, y_blocks))
    max_iterations = 10 * len(blocks)
    print('{} blocks in total'.format(len(blocks)))
    # random_element = blocks.pop(random.randint(0, len(blocks) - 1))

    train, val, test = [], [], []
    train_distr, val_distr, test_distr= (
        {0: 0, 1: 0, 2: 0, 3: 0} for i in range(3)
    )

    options = [train, val, test]
    options_distr = [train_distr, val_distr, test_distr]

    options, options_distr, blocks = generate_subsets(
        blocks,
        max_iterations, class_counts, options, options_distr, 0.8, 0.1, eps
    )
    indexes = list(range(len(options)))
    if any(options_distr[indexes[0]].values())==0 or any(options_distr[indexes[1]].values())==0 or any(options_distr[indexes[2]].values())==0:
        print('='*100)
        pass
    else:
        minblocks = len(blocks)
        results = copy.deepcopy(options)
        results_distr = copy.deepcopy(options_distr)
        residual_blocks = copy.deepcopy(blocks)
        break

  0%|          | 0/30 [00:00<?, ?it/s]

2250 blocks in total
Final distribution of classes: [{0: 0.777240607763555, 1: 0.013123634093037777, 2: 0.06278124674784057, 3: 0.14685451139556666}, {0: 0.8084422758239734, 1: 0.010145000920640766, 2: 0.05526284293868532, 3: 0.12614988031670044}, {0: 0.8078050481988838, 1: 0.008483003551496702, 2: 0.06707191780821918, 3: 0.11664003044140031}]


  0%|          | 0/30 [00:48<?, ?it/s]


In [12]:
options_distr[indexes[0]].values()

dict_values([1493701, 25221, 120653, 282225])

In [13]:
# Check residuals
for i in range(len(blocks)):
    print("Block ", i)
    print(np.unique(residual_blocks[i][1].flatten(), return_counts=True)[1])

Block  0
[7069 1251 1680]
Block  1
[9885   97   18]
Block  2
[9872   88   40]
Block  3
[5000]
Block  4
[5000]
Block  5
[9977   20    3]
Block  6
[4643  105  338 4914]
Block  7
[1843 1631  488 6038]
Block  8
[10000]
Block  9
[6443  359 3198]
Block  10
[3627  277 1349 4747]
Block  11
[10000]
Block  12
[9431  522   47]
Block  13
[10000]
Block  14
[3370  727 2759 3144]
Block  15
[5000]
Block  16
[9775  225]
Block  17
[9984   14    2]
Block  18
[10000]
Block  19
[5853 1124 3023]
Block  20
[7815   50  588 1547]
Block  21
[8405    6  698  891]
Block  22
[9805  160   35]
Block  23
[1655 3113   89 5143]
Block  24
[9875  125]
Block  25
[4626    8 1039 4327]
Block  26
[10000]
Block  27
[9968   15   17]
Block  28
[2790 2259  391 4560]
Block  29
[9992    8]
Block  30
[8269  125 1606]
Block  31
[9561  359   80]
Block  32
[9599   93  308]
Block  33
[10000]
Block  34
[8548    1  710  741]
Block  35
[9985   15]
Block  36
[10000]
Block  37
[10000]
Block  38
[10000]
Block  39
[10000]
Block  40
[10000]
Bl

In [14]:
# Check results distr
for s, set in enumerate(["train", "val", "test"]):
    print(set, [results_distr[s][i] / class_counts[i] for i in range(4)])

train [0.08502639846757908, 0.10512341716753223, 0.09529191456559735, 0.1027713768833681]
val [0.09997212465725766, 0.09186055235538809, 0.09481803369450954, 0.09979374715965368]
test [0.07250588971212006, 0.05575238206387182, 0.0835286117422941, 0.06697318130440355]


In [15]:
X_train = np.concatenate([block[0].reshape(-1, len(X_keys)) for block in options[0]], axis=0)
X_val = np.concatenate([block[0].reshape(-1, len(X_keys)) for block in options[1]], axis=0)
X_test = np.concatenate([block[0].reshape(-1, len(X_keys)) for block in options[2]], axis=0)

In [16]:
y_train = np.concatenate([block[1].reshape(-1, 1) for block in options[0]], axis=0)
y_val = np.concatenate([block[1].reshape(-1, 1) for block in options[1]], axis=0)
y_test = np.concatenate([block[1].reshape(-1, 1) for block in options[2]], axis=0)

### Target one hot encoding and Train/test split

In [19]:
# read data and apply one-hot encoding
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False).fit(y_train)
y_train = ohe.transform(y_train)
y_val = ohe.transform(y_val)
y_test = ohe.transform(y_test)

# Define scaler based on whole dataset
scaler = MinMaxScaler()
minmax = scaler.fit(X_train)
joblib.dump(minmax, os.path.join(path_to_npys_data, "scaler_D.save"))

# Normalization using minmax scaler
X_train = minmax.transform(X_train)
X_val = minmax.transform(X_val)
X_test = minmax.transform(X_test)
X_holdout = minmax.transform(X_holdout)

X = dict()
y = dict()

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("X_holdout shape:", X_holdout.shape)

X_train shape: (16870800, 164)
X_val shape: (2141200, 164)
X_test shape: (2127700, 164)


In [20]:
X["Train"] = X_train
X["Val"] = X_val
X["Test"] = X_test
y["Train"] = y_train
y["Val"] = y_val
y["Test"] = y_test

In [112]:
# # save holdout data 
# with open(os.path.join("..", "data", "processed_files", "pkls", "X_holdout.pkl"), 'wb') as file:
#     pickle.dump(X_holdout, file)
    
# with open(os.path.join("..", "data", "processed_files", "pkls", "y_holdout.pkl"), 'wb') as file:
#     pickle.dump(y_holdout, file)

In [21]:
# save dictionary pkl file
with open(os.path.join("..", "data", "processed_files", "pkls", "X_D_1.pkl"), "wb") as fp:
    pickle.dump(X, fp)

with open(os.path.join("..", "data", "processed_files", "pkls", "y_D_1.pkl"), "wb") as fp:
    pickle.dump(y, fp)

In [22]:
X["Train"] = reshape_data(pd.DataFrame(X_train, columns=X_keys))
X["Val"] = reshape_data(pd.DataFrame(X_val, columns=X_keys))
X["Test"] = reshape_data(pd.DataFrame(X_test, columns=X_keys))

In [23]:
with open(os.path.join("..", "data", "processed_files", "pkls", "X_lstm_D.pkl"), "wb") as fp:
    pickle.dump(X, fp)

with open(os.path.join("..", "data", "processed_files", "pkls", "y_lstm_D.pkl"), "wb") as fp:
    pickle.dump(y, fp)